# Ransomware IR log verification & correlations (MDE + Sysmon + Cloudflare)

This notebook contains:
- A **non-overlapping** list of log types needed to verify ransomware activity
- Correlation concepts and an **offline workflow** if you export results from Sentinel/Splunk

Use it as an IR lead runbook artifact to support containment and scoping decisions.


## 1) Logs required (each category has a single purpose)

1. **Endpoint execution & impact**
   - Confirms payload execution chain and encryption behavior.
   - Sources: MDE tables + Sysmon (1/11/3/22/13)

2. **Authentication & privilege changes**
   - Confirms credential abuse and escalation.
   - Sources: Windows Security (4624/4625/4672/4720/4728/4732/1102) + MDE `DeviceLogonEvents`

3. **Remote execution channels**
   - Confirms lateral movement using services/tasks/remote tooling.
   - Sources: 7045 (System), 4698 (Security), plus MDE/Sysmon process telemetry

4. **Network/exfil visibility (Cloudflare)**
   - Confirms suspicious downloads/C2-like behavior and upload/exfil patterns.
   - Sources: Cloudflare Gateway/DNS/Access/WAF/Audit (Sentinel: CommonSecurityLog or custom table)

5. **Backup/recovery tampering**
   - Confirms recovery inhibition attempts.
   - Sources: process telemetry for vssadmin/wmic/wbadmin/bcdedit/wevtutil + backup platform logs if available


## 2) Correlation query library (copy/paste to SIEM)

This notebook intentionally does **not** duplicate the full query blocks; they live in the repo `README.md` so there is one authoritative location.
- Section **A**: Impact confirmation (file spike)
- Section **B**: Recovery inhibition (backup delete/log clear)
- Section **C**: Remote exec channel (service/task)
- Section **D**: Credential abuse (logon burst)
- Section **E**: Privilege change (group add)
- Section **F**: Cloudflare upload-like activity


## 3) Offline workflow (if you export CSV results)

Recommended exports (CSV):
- `file_spike.csv`  — output of correlation A
- `backup_delete.csv` — output of correlation B
- `service_task.csv` — output of correlation C
- `logon_burst.csv` — output of correlation D
- `priv_group_change.csv` — output of correlation E
- `cloudflare_upload.csv` — output of correlation F


In [ ]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path("data")

def load_csv(name):
    p = DATA_DIR / name
    if not p.exists():
        print(f"[!] Missing: {p}")
        return pd.DataFrame()
    df = pd.read_csv(p)
    # normalize common time columns
    for c in ("Timestamp","TimeGenerated","_time","timestamp"):
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce", utc=True)
    return df

file_spike = load_csv("file_spike.csv")
backup_delete = load_csv("backup_delete.csv")
cloudflare_upload = load_csv("cloudflare_upload.csv")

(len(file_spike), len(backup_delete), len(cloudflare_upload))


## 4) Time-window correlation template (encryption spike ↔ Cloudflare uploads)

This is the core “chain” correlation:
- find a host with encryption-like file activity,
- map it to IP (from MDE `DeviceNetworkEvents` or DHCP),
- then check Cloudflare upload-like activity within a window around the spike.


In [ ]:
import pandas as pd

def correlate_time_window(left, right, left_time, right_time, left_key, right_key, minutes=60):
    if left.empty or right.empty:
        return pd.DataFrame()
    l = left.copy()
    r = right.copy()
    l[left_time] = pd.to_datetime(l[left_time], errors="coerce", utc=True)
    r[right_time] = pd.to_datetime(r[right_time], errors="coerce", utc=True)

    joined = l.merge(r, left_on=left_key, right_on=right_key, how="inner", suffixes=("_enc","_cf"))
    w = pd.Timedelta(minutes=minutes)
    joined = joined[(joined[right_time] >= joined[left_time] - w) & (joined[right_time] <= joined[left_time] + w)]
    return joined

# Example (rename columns to match your exports):
# correlated = correlate_time_window(file_spike, cloudflare_upload,
#                                   left_time="Timestamp", right_time="TimeGenerated",
#                                   left_key="LocalIP", right_key="SourceIP", minutes=60)
# correlated.head()
